In [1]:
import pandas as pd
import requests
from io import BytesIO
import warnings
import os 

In [2]:
year = '2020'
months = ['10', '11', '12']
full_df_parquet = pd.DataFrame()

In [ ]:
for mon in months: 
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{mon}.parquet'
    r = requests.get(url)
    df_parquet = pd.read_parquet(BytesIO(r.content))
    
    full_df_parquet = pd.concat([full_df_parquet, df_parquet], ignore_index=True)
    print(df_parquet.shape, full_df_parquet.shape)

In [3]:

full_df_csv = pd.DataFrame()
for mon in months: 
    url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2020-{mon}.csv.gz'
    r = requests.get(url)
    df_csv =  pd.read_csv(BytesIO(r.content), compression='gzip')
    
    full_df_csv = pd.concat([full_df_csv, df_csv], ignore_index=True)
    print(df_csv.shape, full_df_csv.shape)

/tmp/ipykernel_226166/3054587626.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv =  pd.read_csv(BytesIO(r.content), compression='gzip')


(95120, 20) (95120, 20)


/tmp/ipykernel_226166/3054587626.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv =  pd.read_csv(BytesIO(r.content), compression='gzip')


(88605, 20) (183725, 20)
(83130, 20) (266855, 20)


/tmp/ipykernel_226166/3054587626.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv =  pd.read_csv(BytesIO(r.content), compression='gzip')


In [4]:
data = full_df_csv.copy()
data.columns  

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [9]:

data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266855 entries, 0 to 266854
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   vendorid               147393 non-null  float64
 1   lpep_pickup_datetime   266855 non-null  object 
 2   lpep_dropoff_datetime  266855 non-null  object 
 3   store_and_fwd_flag     147393 non-null  object 
 4   ratecodeid             147393 non-null  float64
 5   pulocationid           266855 non-null  int64  
 6   dolocationid           266855 non-null  int64  
 7   passenger_count        147393 non-null  float64
 8   trip_distance          266855 non-null  float64
 9   fare_amount            266855 non-null  float64
 10  extra                  266855 non-null  float64
 11  mta_tax                266855 non-null  float64
 12  tip_amount             266855 non-null  float64
 13  tolls_amount           266855 non-null  float64
 14  ehail_fee              0 non-null   

In [10]:
data_0 = data[(data['passenger_count'] == 0) & (data['trip_distance'] == 0)]
data_0.head()

,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pulocationid,dolocationid,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
8303,2.0,2020-10-05 21:29:21,2020-10-05 21:30:54,N,5.0,189,189,0.0,0.0,8.50,0.0,0.0,1.76,0.0,NaN,0.3,10.56,1.0,2.0,0.0
13407,2.0,2020-10-08 21:30:05,2020-10-08 21:31:13,N,5.0,41,41,0.0,0.0,0.11,0.0,0.0,0.00,0.0,NaN,0.3,0.41,2.0,2.0,0.0
13443,2.0,2020-10-08 22:55:41,2020-10-08 22:56:41,N,5.0,74,74,0.0,0.0,0.01,0.0,0.0,0.00,0.0,NaN,0.3,0.31,2.0,2.0,0.0
23023,2.0,2020-10-15 02:07:50,2020-10-15 02:07:53,N,5.0,41,264,0.0,0.0,10.00,0.0,0.0,0.00,0.0,NaN,0.3,10.30,1.0,2.0,0.0
28464,2.0,2020-10-18 00:37:37,2020-10-18 00:37:40,N,5.0,181,181,0.0,0.0,15.00,0.0,0.0,0.00,0.0,NaN,0.3,15.30,1.0,2.0,0.0


In [12]:
data = data[(data['passenger_count'] > 0) & (data['trip_distance'] > 0)]
data.shape

(139370, 20)

In [13]:
data.lpep_pickup_datetime= pd.to_datetime(data.lpep_pickup_datetime)
data.lpep_dropoff_datetime= pd.to_datetime(data.lpep_dropoff_datetime)

In [14]:

data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt.date



In [15]:
value_counts= data['lpep_pickup_date'].value_counts()
value_counts
for value in value_counts.index:
    print(value)

2020-10-30
2020-11-12
2020-11-20
2020-11-13
2020-10-29
2020-12-11
2020-12-04
2020-10-15
2020-11-02
2020-10-23
2020-12-03
2020-11-10
2020-12-01
2020-10-01
2020-10-16
2020-11-06
2020-11-05
2020-10-22
2020-12-23
2020-10-02
2020-12-02
2020-12-18
2020-12-15
2020-12-10
2020-10-31
2020-11-19
2020-12-09
2020-10-09
2020-11-25
2020-10-28
2020-11-23
2020-10-14
2020-10-27
2020-11-18
2020-12-08
2020-11-24
2020-10-26
2020-11-11
2020-10-08
2020-11-16
2020-10-21
2020-10-03
2020-10-19
2020-11-09
2020-11-21
2020-10-24
2020-10-17
2020-10-10
2020-11-03
2020-12-14
2020-10-13
2020-12-07
2020-11-17
2020-10-07
2020-10-05
2020-11-04
2020-10-20
2020-12-21
2020-12-22
2020-10-06
2020-11-07
2020-12-16
2020-11-14
2020-12-24
2020-11-30
2020-12-19
2020-12-31
2020-12-12
2020-12-29
2020-11-28
2020-12-05
2020-11-27
2020-12-30
2020-12-28
2020-10-12
2020-10-18
2020-11-01
2020-11-08
2020-10-25
2020-10-04
2020-12-13
2020-10-11
2020-12-26
2020-11-22
2020-11-29
2020-12-06
2020-11-26
2020-11-15
2020-12-20
2020-12-27
2020-12-25

In [ ]:
value_counts= data['vendorid'].value_counts()
value_counts